In [1]:
%load_ext autoreload
%autoreload 2

## Application de modèle de co-referencial aspects identification

### Importation des bibliothèque et le modèle Word2Vec

In [1]:
import os
os.chdir("..")

In [2]:
from Aspects.CoRefAspectIdentGrouping import CoRefAspectIdentGrouping
from gensim.models import Word2Vec
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
file = open('explicit_aspects.txt', 'r')
explicit_aspects = eval(file.readline())
file.close()

In [5]:
data = pd.Series([0])

In [6]:
co_ref_id_grop = CoRefAspectIdentGrouping(data, dict(explicit_aspects), nlp)
co_ref_id_grop.model_wv = Word2Vec.load("100K_reviews_model_sg_hs_10.pkl")

In [7]:
co_ref_id_grop.get_co_reference_aspects_groups(threshold=0.4981)

{'breakfast': ['cafe',
  'restaurant',
  'breakfast',
  'food',
  'shop',
  'buffet',
  'dinner',
  'bar',
  'snack',
  'drink'],
 'night': ['night'],
 'reception': ['lobby', 'desk', 'reception'],
 'access': ['access'],
 'time': ['time'],
 'minibar': ['minibar'],
 'metro': ['airport', 'bus', 'metro', 'station'],
 'attraction': ['attraction'],
 'quality': ['quality'],
 'area': ['area'],
 'facility': ['facility'],
 'member': ['member'],
 'distance': ['distance'],
 'view': ['garden', 'courtyard', 'balcony', 'view'],
 'manager': ['manager'],
 'stay': ['stay', 'trip', 'experience'],
 'coffee': ['coffee'],
 'check': ['check'],
 'hotel': ['location', 'place', 'hotel'],
 'tv': ['tv'],
 'water': ['water'],
 'tower': ['tower'],
 'street': ['street'],
 'price': ['price'],
 'luggage': ['luggage'],
 'staff': ['staff', 'customer', 'service'],
 'parking': ['parking'],
 'towel': ['towel'],
 'room': ['bathroom', 'bed', 'shower', 'room'],
 'money': ['money']}

In [8]:
co_ref_id_grop.model_wv.wv.most_similar('food', topn=10)

[('meals', 0.6888847947120667),
 ('foods', 0.6066862344741821),
 ('meal', 0.5863748788833618),
 ('buffet', 0.5842719078063965),
 ('restaurant', 0.5658935904502869),
 ('breakfast', 0.5546433925628662),
 ('varied', 0.550567090511322),
 ('tasty', 0.5496057868003845),
 ('menu', 0.5426717400550842),
 ('dinner', 0.5335071682929993)]

In [9]:
co_ref_id_grop.model_wv.wv.most_similar('buffet', topn=10)

[('breakfast', 0.747629702091217),
 ('continental', 0.7096893191337585),
 ('foods', 0.6526040434837341),
 ('choices', 0.6445614099502563),
 ('buffett', 0.6417891383171082),
 ('spread', 0.6320570707321167),
 ('buffets', 0.6218664646148682),
 ('varied', 0.6136159300804138),
 ('selections', 0.6078395843505859),
 ('variety', 0.5999879837036133)]

In [10]:
co_ref_id_grop.model_wv.wv.most_similar('staff', topn=10)

[('polite', 0.7135059237480164),
 ('staffs', 0.6921836733818054),
 ('professional', 0.6821528077125549),
 ('courteous', 0.6530322432518005),
 ('personnel', 0.6478990912437439),
 ('employees', 0.6445193290710449),
 ('receptionists', 0.642317533493042),
 ('attentive', 0.6371670961380005),
 ('friendly', 0.6248290538787842),
 ('helpful', 0.6148644685745239)]

In [11]:
co_ref_id_grop.model_wv.wv.most_similar('bus', topn=10)

[('buses', 0.7680094242095947),
 ('tram', 0.7403375506401062),
 ('stops', 0.702381432056427),
 ('train', 0.6893447637557983),
 ('airport', 0.6671081781387329),
 ('taxi', 0.6599125266075134),
 ('x95', 0.6518966555595398),
 ('haneda', 0.6444940567016602),
 ('narita', 0.6289826035499573),
 ('metro', 0.617334246635437)]

In [12]:
co_ref_id_grop.model_wv.wv.most_similar('hotel', topn=10)

[('resort', 0.594900369644165),
 ('place', 0.590297281742096),
 ('property', 0.5601755976676941),
 ('sukhothai', 0.5587882995605469),
 ('establishment', 0.5412606596946716),
 ("'", 0.515845775604248),
 ('however', 0.45468708872795105),
 ('-', 0.45446643233299255),
 ('motel', 0.4521563947200775),
 ('facility', 0.44720637798309326)]